In [2]:
import pandas as pd
import numpy as np

In [44]:
crashes = pd.read_csv("../data/dwh/CrashesFact.csv")
drivers = pd.read_csv("../data/dwh/DriversFact.csv")
roaddim = pd.read_csv("../data/dwh/RoadDim.csv")
vehicles = pd.read_csv("../data/dwh/VehicleDim.csv")
location = pd.read_csv("../data/dwh/LocationAreaDim.csv")
weather = pd.read_csv("../data/dwh/WeatherFact.csv")
datehour = pd.read_csv("../data/dwh/DateHourDim.csv")

In [39]:
fact_merged = pd.read_csv("../data/etl_out/VehicleCrashFact.csv")
roaddim = pd.read_csv("../data/etl_out/RoadDim.csv")
vehicles = pd.read_csv("../data/etl_out/VehicleDim.csv")
location = pd.read_csv("../data/etl_out/LocationAreaDim.csv")
weather = pd.read_csv("../data/etl_out/WeatherFact.csv")
datehour = pd.read_csv("../data/etl_out/DateHourDim.csv")
merged = pd.read_csv("../data/etl_out/MergedData.csv")

In [4]:
len(drivers)

172105

In [28]:
fact_merged = drivers.merge(crashes, on='ReportNumber')

NameError: name 'drivers' is not defined

In [29]:
len(fact_merged)

1596

In [30]:
fact_vehicles = fact_merged.merge(vehicles, on='VehicleKey')

In [31]:
len(fact_vehicles)

1596

In [32]:
all = fact_vehicles.merge(roaddim, left_on='RoadKey', right_on='RoadKey').merge(roaddim, left_on='CrossStreetKey', right_on='RoadKey')

In [33]:
len(all)

1596

In [34]:
withloc = all.merge(location, on='LocationAreaKey')

In [35]:
len(withloc)

1596

In [36]:
withweather = withloc.merge(weather, how='left', on=['LocationAreaKey', 'DateHourKey'])

In [37]:
len(withweather)

1596

In [40]:
len(merged)

1587

In [58]:
withtime = withweather.merge(datehour, on='DateHourKey')

In [59]:
withtime

,ReportNumber,VehicleCrashKey,DriverAtFault,DriverInjurySeverity,DriverSubstanceAbuse,DriverDistractedBy,VehicleType,VehicleMovement,VehicleGoingDir,VehicleDamageExtent,...,TimeOfDay,DayNumber,WeekDayNumber,WeekDayName,WeekendFlag,MonthNumber,MonthName,Year_y,HolidayFlag,HolidayName
0,MCP3071002G,91C243A3CF6F413593F9237DCE1A210D,True,NO APPARENT INJURY,NaN,UNKNOWN,PASSENGER,MOVING CONSTANT SPEED,West,DISABLING,...,AM,335,4,Friday,0,12,December,2023,0,NaN
1,MCP3303001S,D095093033AE4B16930822CD54C343AF,True,NO APPARENT INJURY,NONE,NOT DISTRACTED,PASSENGER,MAKING LEFT TURN,South,NO DAMAGE,...,PM,359,0,Monday,0,12,December,2023,1,Christmas Day
2,MCP3313001W,67B0F5E8BDDB46C1ACC8728826322EA3,True,NO APPARENT INJURY,UNKNOWN,UNKNOWN,PASSENGER,MOVING CONSTANT SPEED,West,DESTROYED,...,PM,359,0,Monday,0,12,December,2023,1,Christmas Day
3,MCP2677001V,BF0F265B48D5401D932932D55FCE95DF,False,SUSPECTED SERIOUS INJURY,NaN,NOT DISTRACTED,PASSENGER,MOVING CONSTANT SPEED,South,DESTROYED,...,PM,335,4,Friday,0,12,December,2023,0,NaN
4,MCP2677001V,0122999A342E4C91B6FD182B7725F52A,True,POSSIBLE INJURY,ALCOHOL,UNKNOWN,PASSENGER,CHANGING LANES,South,DISABLING,...,PM,335,4,Friday,0,12,December,2023,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1591,MCP209400N3,50BED956492047558A30BE594146C876,True,NO APPARENT INJURY,NONE,UNKNOWN,PASSENGER,MAKING LEFT TURN,North,DISABLING,...,AM,346,1,Tuesday,0,12,December,2023,0,NaN
1592,MCP32630046,75CC81A1EAF645B09E7FB69A33B3E86D,True,NO APPARENT INJURY,NONE,LOOKED BUT DID NOT SEE,PASSENGER,RIGHT TURN ON RED,West,NO DAMAGE,...,PM,364,5,Saturday,1,12,December,2023,0,NaN
1593,MCP2107004M,797BDB5719D448E0A61FA26B07E4A4B4,True,NO APPARENT INJURY,NONE,LOOKED BUT DID NOT SEE,PASSENGER,SLOWING OR STOPPING,East,FUNCTIONAL,...,AM,336,5,Saturday,1,12,December,2023,0,NaN
1594,MCP30310053,6860304685894D31814EC7EE74EFBBF7,False,POSSIBLE INJURY,NONE,NOT DISTRACTED,PASSENGER,MOVING CONSTANT SPEED,South,DESTROYED,...,PM,346,1,Tuesday,0,12,December,2023,0,NaN


In [44]:
df1 = withweather
df2 = merged

merged_df = df1.merge(df2, on='ReportNumber', how='outer', indicator=True)

# Rows in df1 but not in df2
only_in_df1 = merged_df[merged_df['_merge'] == 'left_only'].drop(columns=['_merge'])

# Rows in df2 but not in df1
only_in_df2 = merged_df[merged_df['_merge'] == 'right_only'].drop(columns=['_merge'])

# Display differences
print("Rows in df1 but not in df2:")
print(only_in_df1)
print("\nRows in df2 but not in df1:")
print(only_in_df2)

Rows in df1 but not in df2:
    ReportNumber                 VehicleCrashKey_x  DriverAtFault_x  \
0    MCP3071002G  91C243A3CF6F413593F9237DCE1A210D             True   
837  MCP2552006V  BB1A5C88135D4581ABC75C1972E3749A            False   
838  MCP2552006V  3B11D65A327A47D7B5AB13838650F4A8             True   
839   DM8370001Z  E39FF23DDA234AA6A890CEA262EFDF30            False   
840   DM8370001Z  51E2A2EF10DC4E79ACF46258E2667B76             True   
841  MCP2552006R  BD0173867072470D88FCAEBA2CF46883             True   
842  MCP2552006R  303C3C2E5FF9401DAC7308C230C6D0F2            False   
843  MCP33220028  6CDCEEC4659342C8967239EF42FACA6B            False   
844  MCP33220028  6A6259D74BD447D88F35ED5C2D6B0319             True   

    DriverInjurySeverity_x DriverSubstanceAbuse_x DriverDistractedBy_x  \
0       NO APPARENT INJURY                    NaN              UNKNOWN   
837     NO APPARENT INJURY                    NaN       NOT DISTRACTED   
838     NO APPARENT INJURY             

In [51]:
only_in_df1.columns

Index(['ReportNumber', 'VehicleCrashKey_x', 'DriverAtFault_x',
       'DriverInjurySeverity_x', 'DriverSubstanceAbuse_x',
       'DriverDistractedBy_x', 'VehicleType_x', 'VehicleMovement_x',
       'VehicleGoingDir_x', 'VehicleDamageExtent_x',
       ...
       'CentroidLatitude_y', 'CentroidLongitude_y', 'Temperature_y',
       'Humidity_y', 'Precipitation_y', 'Rain_y', 'Snow_y', 'WindSpeed_y',
       'WindDirection_y', 'WeatherKey_y'],
      dtype='object', length=147)

In [57]:
weather[weather['DateHourKey'] == 2023120103]

,Temperature,Humidity,Precipitation,Rain,Snow,WindSpeed,WindDirection,LocationAreaKey,DateHourKey,WeatherKey


In [54]:
only_in_df1.loc[0][['DateHourKey_x']]

DateHourKey_x    2023120102
Name: 0, dtype: object

In [48]:
only_in_df1['LocationAreaKey_x']

0      7723271939208770
837                   0
838                   0
839                   0
840                   0
841                   0
842                   0
843                   0
844                   0
Name: LocationAreaKey_x, dtype: int64

In [43]:
df2.dtypes

ReportNumber             object
VehicleCrashKey          object
DriverAtFault              bool
DriverInjurySeverity     object
DriverSubstanceAbuse     object
                         ...   
Rain                    float64
Snow                    float64
WindSpeed               float64
WindDirection           float64
WeatherKey                int64
Length: 74, dtype: object

In [42]:
df1.dtypes

ReportNumber             object
VehicleCrashKey          object
DriverAtFault              bool
DriverInjurySeverity     object
DriverSubstanceAbuse     object
                         ...   
Rain                    float64
Snow                    float64
WindSpeed               float64
WindDirection           float64
WeatherKey              float64
Length: 74, dtype: object

In [43]:
len(np.unique(list(weather['LocationAreaKey'].unique()) + list(withloc['LocationAreaKey'].unique())))

98

In [35]:
weather['LocationAreaKey'].value_counts().sort_values()

LocationAreaKey
7714293738966626    8760
7709647338995261    8760
7709635438994319    8760
7709652138995416    8760
7709648138995515    8760
                    ... 
7743621739190007    8760
7719017539329336    8760
7716365739170519    8760
7718114539090864    8760
7731995139281675    8760
Name: count, Length: 97, dtype: int64

In [31]:
withweather[withweather['WeatherKey'].isna()]

,ReportNumber,VehicleCrashKey,DriverAtFault,DriverInjurySeverity,DriverSubstanceAbuse,DriverDistractedBy,VehicleType,VehicleMovement,VehicleGoingDir,VehicleDamageExtent,SpeedLimit,ParkedVehicle,SubstanceAbuseContributed,VehiclesCrashedTotal,VehicleKey,LocalCaseNumber,AgencyName,ACRSReportType,HitRun,LaneDirection,LaneNumber,NumberOfLanes,RoadGrade,NonTraffic,OffRoadIncident,AccidentAtFault,CollisionType,SurfaceCondition,Light,TrafficControl,Junction,IntersectionType,RoadAlignment,RoadCondition,RoadDivision,Latitude,Longitude,RoadKey_x,CrossStreetKey,NonMotoristTotal,NonMotoristInjury,NonMotoristFatal,DateHourKey,LocationAreaKey,Make,Year,BaseModel,BodyClass,Cylinders,Displacement,Transmission,Drivetrain,FuelType,CityMPG,HighwayMPG,RoadName_x,RouteType_x,RoadName_y,RouteType_y,RoadKey_y,Zipcode,MailCity,ShapeLength,ShapeArea,CentroidLatitude,CentroidLongitude,Temperature,Humidity,Precipitation,Rain,Snow,WindSpeed,WindDirection,WeatherKey
0,MCP3170003V,4E492574893B4EB1ADCA53FDD633D6C4,True,NO APPARENT INJURY,NONE,LOOKED BUT DID NOT SEE,PASSENGER,MOVING CONSTANT SPEED,South,FUNCTIONAL,0,False,False,2,6880218977007057,240000438,Montgomery County Police,Property Damage,False,UNKNOWN,0,0,UNKNOWN,True,True,DRIVER,OPPOSITE DIRECTION SIDESWIPE,UNKNOWN,DAYLIGHT,NO CONTROLS,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,39.165005,-77.249310,7781945108562677,7781945108562677,0,0,0,2024010314,7729807639132189,Lexus,2017,LS,Midsize Cars,8.0,4.6,Automatic 8,AWD,Premium Gasoline,16,23,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,7781945108562677,20874,GERMANTOWN,213170.688025,6.504704e+08,39.132190,-77.298076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MCP3170003V,904F21142B5646D9B105356073FA1D8A,True,NO APPARENT INJURY,NONE,LOOKED BUT DID NOT SEE,BUS,MOVING CONSTANT SPEED,North,NO DAMAGE,0,False,False,2,2250767647570290,240000438,Montgomery County Police,Property Damage,False,UNKNOWN,0,0,UNKNOWN,True,True,DRIVER,OPPOSITE DIRECTION SIDESWIPE,UNKNOWN,DAYLIGHT,NO CONTROLS,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,39.165005,-77.249310,7781945108562677,7781945108562677,0,0,0,2024010314,7729807639132189,Freightliner,0,Unknown,NaN,0.0,0.0,NaN,NaN,NaN,0,0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,7781945108562677,20874,GERMANTOWN,213170.688025,6.504704e+08,39.132190,-77.298076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MCP2641000V,66DE0D5550734970912B547997960520,True,NO APPARENT INJURY,NONE,UNKNOWN,EMERGENCY,PARKING,UNKNOWN,SUPERFICIAL,0,False,False,1,2250767647570290,170509707,Montgomery County Police,Property Damage,False,UNKNOWN,0,0,UNKNOWN,True,True,DRIVER,SINGLE VEHICLE,UNKNOWN,DARK LIGHTS ON,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,39.185527,-77.260902,7781945108562677,7781945108562677,0,0,0,2017052422,7729807639132189,Freightliner,0,Unknown,NaN,0.0,0.0,NaN,NaN,NaN,0,0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,7781945108562677,20874,GERMANTOWN,213170.688025,6.504704e+08,39.132190,-77.298076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,MCP9106001C,F3D29BD1E3BD4381B84BC20972C35BCC,False,NO APPARENT INJURY,NONE,NOT DISTRACTED,BUS,PARKED,UNKNOWN,SUPERFICIAL,0,True,False,2,2250767647570290,170509800,Montgomery County Police,Property Damage,False,UNKNOWN,0,0,UNKNOWN,True,True,DRIVER,SAME DIRECTION RIGHT TURN,UNKNOWN,DAYLIGHT,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,39.175580,-77.265003,7781945108562677,7781945108562677,0,0,0,2017052511,7729807639132189,Freightliner,0,Unknown,NaN,0.0,0.0,NaN,NaN,NaN,0,0,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,7781945108562677,20874,GERMANTOWN,213170.688025,6.504704e+08,39.132190,-77.298076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,MCP2496002Z,172DA96A87CD47CBA2AC9ECA9F055EFF,True,POSSIBLE INJURY,NONE,NOT DISTRACTED,TRUCK,BACKING,East,UNKNOWN,15,False,False,1,2250767647570290,210029371,Montgomery County Police,Injury,False,UNKNOWN,0,0,UNKNOWN,True,True,DRIVER,SINGLE VEHICLE,UNKNOWN,DAYLIGHT,NO CONTROLS,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,39.074756,-77.329079,7781945108562677,7781945108562677,0,0,0,2021072908,7729807639132189,Freightliner,0,Unknown,NaN,0.0,0.0,NaN,NaN,NaN,0,0,UNKNOWN,UN

In [21]:
# fact_merged['DateHourKey'] = fact_merged['DateHourKey'].astype(str)
filtered_df = fact_merged[fact_merged['DateHourKey'].astype(str).str.startswith('2023')]

In [23]:
filtered_df.to_csv("../data/dwh/VehicleCrashFact2023.csv")

In [30]:
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [51]:
crashes = pd.read_csv("../data/etl_out/VehicleCrashFact.csv")
roaddim = pd.read_csv("../data/etl_out//RoadDim.csv")
vehicles = pd.read_csv("../data/dwh/VehicleDim.csv")

C:\Users\tymek\AppData\Local\Temp\ipykernel_14004\2494988043.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  crashes = pd.read_csv("../data/etl_out/VehicleCrashFact.csv")


In [46]:
len(crashes)

18885

In [47]:
meee = crashes.merge(weather, on=['LocationAreaKey', 'DateHourKey'])

In [49]:
# czegoś brakuje ...

In [48]:
len(meee)

18859

In [52]:
len(crashes.merge(vehicles, on='VehicleKey'))

18885